BCH Code (Bose-Chaudhuri-Hocquenghem Codes):

Коди BCH - це циклічні блочні коди, які використовуються для виявлення та виправлення помилок у передачі даних. Вони є розширенням кодів Хеммінга та призначені для виявлення та виправлення більшої кількості помилок, включаючи відомий клас кодів Хеммінга.

Кодування BCH:
Вибір параметрів: Визначте параметри коду BCH, такі як довжина коду (n), кількість перевірочних бітів (m), і відстань виправлення помилок (t).
Генерація породжуючого полінома: На основі параметрів коду згенеруйте породжуючий поліном. Зазвичай це робиться за допомогою примітивних елементів у розширеному полі Галуа GF(2^m).

Кодування повідомлення: Розбийте початкове повідомлення на блоки довжиною k, де k = n - m. Додайте до кожного блоку перевірочні біти, обчислені як залишок від ділення полінома, що представляє блок, на породжуючий поліном.

Формування кодового слова: Зіберіть кодове слово, об'єднуючи початкові блоки і відповідні перевірочні біти.

Декодування BCH:

Отримання прийнятого слова: Отримайте прийняте слово, включаючи інформаційні та перевірочні біти.

Синдромний декодування: Обчисліть синдром для прийнятого слова, використовуючи прийняті перевірочні біти та породжуючий поліном. Якщо синдром нульовий, то немає помилок і повідомлення вважається коректно прийнятим. У протилежному випадку переходьте до кроку 3.

Визначення місця помилок: Використовуйте синдром для визначення місця помилок у кодовому слові.

Пошук та виправлення помилок: Використовуйте алгоритми виправлення помилок, такі як метод Чейза, для пошуку та виправлення помилок у кодовому слові.

Відновлення початкового повідомлення: Після виправлення помилок видаліть перевірочні біти та відновіть початкові блоки повідомлення.

In [3]:
def generate_generator_polynomial(m, t):
    polynomials = [[1] + [0] * i for i in range(m)]

    for i in range(1, t):
        new_polynomials = []
        for poly in polynomials:
            for j in range(m):
                if j >= len(poly):
                    continue
                new_poly = poly.copy()
                new_poly[j] = 1
                if new_poly not in new_polynomials:
                    new_polynomials.append(new_poly)

        polynomials.extend(new_polynomials)

    
    polynomial_values = [
        (poly, int("".join(map(str, poly)), 2)) for poly in polynomials
    ]
    
    generator_polynomials = []
    
    for _, elem in enumerate(polynomial_values):
        if elem[1] >= 2 * t + 1:
            generator_polynomials.append(elem[0])

    print(generator_polynomials)
    return generator_polynomials[0]

n = 4
t = 2

generator_polynomial = generate_generator_polynomial(n, t)
print(
    "Породжуючий поліном для BCH коду з n = {} та t = {}: {}".format(
        n, t, generator_polynomial
    )
)

[[1, 0, 0, 0], [1, 1, 0], [1, 0, 1], [1, 0, 0, 0], [1, 1, 0, 0], [1, 0, 1, 0], [1, 0, 0, 1]]
Породжуючий поліном для BCH коду з n = 4 та t = 2: [1, 0, 0, 0]


In [4]:
def to_string(polynomial):
    str_polynom = ""
    for i in range(len(polynomial)):
        str_polynom += str(polynomial[i])
    return str_polynom

print(generator_polynomial)
print(to_string(generator_polynomial))

[1, 0, 0, 0]
1000


In [13]:
def align_size(dividend, divisor):
    # Визначаємо довжини дивідента та дивізора
    dividend_length = len(dividend)
    divisor_length = len(divisor)

    # Знаходимо максимальну довжину
    max_length = max(dividend_length, divisor_length)

    # Вирівнюємо довжини дивідента та дивізора, додаючи нулі до початку меншого
    if dividend_length < max_length:
        diff = max_length - dividend_length
        dividend = "0" * diff + dividend
    elif divisor_length < max_length:
        diff = max_length - divisor_length
        divisor = "0" * diff + divisor

    return dividend, divisor


dividend = "01"
divisor = to_string(generator_polynomial)

print(align_size(dividend, divisor))

('0001', '1000')


In [23]:
dividend = "01"
divisor = to_string(generator_polynomial)

dividend, divisor = align_size(dividend, divisor)

def binary_division(dividend, divisor):
    dividend_length = len(dividend)
    divisor_length = len(divisor)

    # Проводимо ділення за допомогою операції XOR
    chastka = ""
    current_dividend = dividend[
        :divisor_length
    ]  # Починаємо з перших n символів дивідента
    for i in range(divisor_length, dividend_length):
        # Додаємо нуль до частки, якщо не можемо виконати ділення
        if current_dividend[0] == "0":
            chastka += "0"
        else:
            chastka += "1"
            # Виконуємо операцію XOR
            current_dividend = "".join(
                str(int(current_dividend[j]) ^ int(divisor[j]))
                for j in range(divisor_length)
            )
        # Додаємо наступний біт дивідента
        current_dividend += dividend[i]

    # Останнє ділення
    if current_dividend[0] == "0":
        chastka += "0"
    else:
        chastka += "1"

    # Отримуємо залишок (останні n-1 біт дивідента)
    zalyshok = current_dividend[-1]

    return chastka, zalyshok


chastka, zalyshok = binary_division(dividend, divisor)
print("Частка:", chastka)
print("Залишок:", zalyshok)

Частка: 0
Залишок: 1


In [31]:
def encoding(k, message, polynomial):
    encoded_message = ""
    blocks = [message[i : i + k] for i in range(0, len(message), k)]
    # print(blocks)
    checking_bits = []
    for i in range(len(blocks)):
        checking_bits.append(binary_division(blocks[i], polynomial)[1])
    # print(checking_bits)

    for i in range(len(blocks)):
        encoded_message += blocks[i]
        encoded_message += checking_bits[i]

    return encoded_message

k = 2
message = "010001"
pol = "1000"
print(encoding(k, message, pol))

011000011


In [32]:
import random

def add_errors(sequence, error_probability):
    # Перетворюємо рядок у список символів для зручності модифікації
    sequence_list = list(sequence)

    # Проходимо по кожному символу у послідовності
    for i in range(len(sequence_list)):
        # Генеруємо випадкове число для визначення, чи потрібно змінити даний біт
        if random.random() < error_probability:
            # Змінюємо символ на протилежний: '0' на '1' або '1' на '0'
            sequence_list[i] = "0" if sequence_list[i] == "1" else "1"

    # Повертаємо модифіковану послідовність у форматі рядка
    return "".join(sequence_list)


# Приклад використання:
original_sequence = "011000011"
error_probability = 0.3  # Ймовірність помилки - 10%

# Додаємо помилки до повідомлення
noisy_sequence = add_errors(original_sequence, error_probability)
print("Початкове повідомлення:", original_sequence)
print("Повідомлення з помилками:", noisy_sequence)

Початкове повідомлення: 011000011
Повідомлення з помилками: 010100011


In [ ]:
def decode():
    pass